In [170]:
import json
import graphviz
from graphviz import Digraph

In [171]:
with open('../testcases/test4/NFA.json') as f:
    nfa = json.load(f)


terminating_states = set()
#loop through the states, and make each transition a list if it isn't already
for state in nfa:
    if state == 'startingState':
        nfa[state] = [nfa[state]]
        continue
    
    for transition in nfa[state]:
        if transition == 'isTerminatingState':
            if nfa[state][transition] == True:
                terminating_states.add(state)
            continue
        if type(nfa[state][transition]) != list:
            nfa[state][transition] = [nfa[state][transition]]

print(nfa)
print(terminating_states)

{'startingState': ['S3'], 'S3': {'isTerminatingState': False, 'epsilon': ['S1', 'S4']}, 'S4': {'isTerminatingState': False, 'epsilon': ['S7']}, 'S1': {'isTerminatingState': False, 'a': ['S2']}, 'S2': {'isTerminatingState': False, 'epsilon': ['S3', 'S4']}, 'S7': {'isTerminatingState': False, 'epsilon': ['S5', 'S8']}, 'S8': {'isTerminatingState': False, 'epsilon': ['S15']}, 'S5': {'isTerminatingState': False, 'b': ['S6']}, 'S6': {'isTerminatingState': False, 'epsilon': ['S7', 'S8']}, 'S15': {'isTerminatingState': False, 'epsilon': ['S13', 'S16']}, 'S16': {'isTerminatingState': True}, 'S13': {'isTerminatingState': False, 'epsilon': ['S11', 'S9']}, 'S14': {'isTerminatingState': False, 'epsilon': ['S15', 'S16']}, 'S11': {'isTerminatingState': False, 'b': ['S12']}, 'S12': {'isTerminatingState': False, 'epsilon': ['S14']}, 'S9': {'isTerminatingState': False, 'a': ['S10']}, 'S10': {'isTerminatingState': False, 'epsilon': ['S14']}}
{'S16'}


## NFA 2 DFA

In [172]:
#global variable to store the closures of each state
#used to avoid recalculating the closure of a state
global_closuers = dict()

def get_possible_inputs(states):
    possible_transitions = set()
    # print(states)
    for state in states:
        for symbol in nfa[state]:
            if symbol != "epsilon" and symbol != "isTerminatingState":
                # print(symbol)
                possible_transitions.add(symbol)
    return possible_transitions

def epsilon_closure(nfa, state, prev_closures):
    # print(state)
    if state in global_closuers:
        return global_closuers[state]

    #initialize the closure of the current state
    # with the current state
    closure = set()
    closure.add(state)
    prev_closures.add(state)
    # add epsilon transitions of the current state
    epsilon_transitions = set(nfa[state].get("epsilon", []))
    
    # print(epsilon_transitions)
    for s in epsilon_transitions:
        #union of the current closure and the closure of the next state
        if s not in prev_closures:
            closure |= epsilon_closure(nfa, s, prev_closures)

    global_closuers[state] = closure
    return closure

def move(nfa, states, symbol):
    result = set()
    for state in states:
        if symbol in nfa[state]:
            if symbol == "epsilon" or symbol == "isTerminatingState":
                continue
            #print(nfa[state][symbol])
            result |= set(nfa[state][symbol])
    return result

def nfa_to_dfa(nfa):
    dfa = {}
    #used to use as a key in dfa, sets are not hashable
    #because they are mutable, but frozensets are immutable
    starting_state = nfa["startingState"][0]
    start_state = frozenset(epsilon_closure(nfa, starting_state, set()))
    is_terminating = False

    dfa["startingState"] = start_state
    dfa[start_state] = {}
    queue = [start_state]
    while queue:
        current_state = queue.pop(0)
        if current_state & terminating_states:
            is_terminating = True
        dfa[current_state]["isTerminatingState"] = is_terminating
            
        possible_inputs = get_possible_inputs(current_state)
        for minput in possible_inputs:
            possible_transitions = move(nfa, current_state, minput)
            next_states_closures = set()
            for next_state in possible_transitions:
                next_states_closures |= epsilon_closure(nfa, next_state, set())
                
            closure = frozenset(next_states_closures)
            if closure not in dfa:
                dfa[closure] = {}
                queue.append(closure)
            dfa[current_state][minput] = closure
            
    return dfa

dfa = nfa_to_dfa(nfa)
print(dfa)

{'startingState': frozenset({'S9', 'S7', 'S3', 'S5', 'S15', 'S16', 'S4', 'S13', 'S8', 'S11', 'S1'}), frozenset({'S9', 'S7', 'S3', 'S5', 'S15', 'S16', 'S4', 'S13', 'S8', 'S11', 'S1'}): {'isTerminatingState': True, 'a': frozenset({'S9', 'S7', 'S3', 'S2', 'S10', 'S5', 'S15', 'S16', 'S4', 'S14', 'S8', 'S11', 'S13', 'S1'}), 'b': frozenset({'S14', 'S9', 'S7', 'S5', 'S15', 'S16', 'S12', 'S6', 'S8', 'S11', 'S13'})}, frozenset({'S9', 'S7', 'S3', 'S2', 'S10', 'S5', 'S15', 'S16', 'S4', 'S14', 'S8', 'S11', 'S13', 'S1'}): {'isTerminatingState': True, 'a': frozenset({'S9', 'S7', 'S3', 'S2', 'S10', 'S5', 'S15', 'S16', 'S4', 'S14', 'S8', 'S11', 'S13', 'S1'}), 'b': frozenset({'S14', 'S9', 'S7', 'S5', 'S15', 'S16', 'S12', 'S6', 'S8', 'S11', 'S13'})}, frozenset({'S14', 'S9', 'S7', 'S5', 'S15', 'S16', 'S12', 'S6', 'S8', 'S11', 'S13'}): {'isTerminatingState': True, 'a': frozenset({'S15', 'S16', 'S9', 'S14', 'S11', 'S10', 'S13'}), 'b': frozenset({'S14', 'S9', 'S7', 'S5', 'S15', 'S16', 'S12', 'S6', 'S8', 'S1

## Clean States

In [173]:
def clean_DFA(dfa):
    index = 1
    dfa_cleaned = {}
    mapping = {}

    mapping[dfa["startingState"]] = "S0"

    for state in dfa:
        
        if state == "startingState":
            continue

        if mapping.get(state) == None:      
            state_name = "S" + str(index)
            mapping[state] = state_name
            index += 1


    for state in dfa:
        if state == "startingState":
            dfa_cleaned[state] = mapping[dfa[state]]
            continue
        
        dfa_cleaned[mapping[state]] = {"isTerminatingState": dfa[state]["isTerminatingState"]}

        for transition in dfa[state]:
            if transition == "isTerminatingState":
                continue

            dfa_cleaned[mapping[state]][transition] = {mapping[dfa[state][transition]]}

    return dfa_cleaned

dfa_cleaned = clean_DFA(dfa)

## Draw DFA

In [174]:
def draw_dfa(dfa):
    dot = graphviz.Digraph()
    starting_state = dfa['startingState']
    for state, transitions in dfa.items():
        if state == 'startingState':
            continue 
        if state == starting_state:
            dot.node(state, shape='doublecircle' if transitions['isTerminatingState'] else 'circle', color='blue')
        else:
            dot.node(state, shape='doublecircle' if transitions['isTerminatingState'] else 'circle')

        for symbol, next_states in transitions.items():
            if symbol != 'isTerminatingState' and symbol != 'startingState':
                for next_state in next_states:
                    dot.edge(state, next_state, label=symbol)

    dot.render('dfa_graph', format='png', cleanup=True)

In [175]:
# #convert all sets to frozensets
# for state in dfa_cleaned:
#     for transition in dfa_cleaned[state]:
#         if transition == "isTerminatingState":
#             continue
#         dfa_cleaned[state][transition] = frozenset(dfa_cleaned[state][transition])

# for state in dfa:
#     for transition in dfa[state]:
#         if transition == "isTerminatingState":
#             continue
#         dfa[state][transition] = frozenset(dfa[state][transition])

In [176]:
draw_dfa(dfa_cleaned)

## Minimize DFA

In [177]:
def minimize_dfa(dfa):
    #initialize the partition
    partition = [set(), set()]
    for state in dfa:
        if dfa[state]["isTerminatingState"]:
            partition[0].add(state)
        else:
            partition[1].add(state)
    print(partition)

